# Document's Features

#### · Display raw data from cluster

#### · Display raw data and located peaks recorded in cluster 

#### · Find peaks of recorded raw data with the most updated version of the algorithm 

In [1]:
from functions import *

/usr/local/lib/python3.9/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#!pip install pymongo==3.13.0

In [3]:
#import util

In [4]:
my_name = 'braulio.ramirez'

In [5]:
logger = logging.getLogger('jhub')
hostname = 'mongos.mongo.svc.cluster.local:27017'
pemkeyfile = '/etc/mongo/jhub-keypem.pem'
sslca = '/etc/mongo/root-ca.pem'

nirscloud_util_meta.init(logger, 'meta', hostname=hostname, ssl=True, cert=pemkeyfile, ca=sslca)

In [6]:
spark_kerberos_principal = my_name + '@BABYNIRS.ORG'

params = {
    'spark_kerberos_principal': spark_kerberos_principal,
}
nirscloud_util_hdfs.init('/etc/jhub/conf/production.ini', params=params)

2023-07-07 20:27:38 [INFO ] cfg#_init_hdfs_kinit@79: after hdfs_kinit: the_stdout: b'' the_stderr: b''
2023-07-07 20:27:38 [INFO ] client#__init__@192: Instantiated <KerberosClient(url='https://hdfs2.babynirs.org:9870;https://hdfs1.babynirs.org:9870;https://hdfs4.babynirs.org:9870')>.


### Pulling the recorded peaks from the cluster

In [7]:
hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'

In [8]:
kafka_topics = 'nk_rpeak2_NICU'
hdfs_prefix = '/kafka/topics/%s'% (kafka_topics)

In [9]:
def _get_df(hdfs_path, the_id):
    hdfs_path = hdfs_path
    full_path = nirscloud_util_hdfs.full_path(hdfs_prefix, hdfs_path)
    err, df = nirscloud_util_hdfs.from_hdfs_path(full_path)
    print(err, df)
    is_valid = df['id'] == the_id
    df = df[is_valid]
    return df

In [10]:
df_II_peaks = _get_df(hdfs_path, 'II')

2023-07-07 20:27:41 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:27:42 [INFO ] client#status@320: Fetching status for '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-07-07 20:27:43 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:27:45 [INFO ] client#status@320: Fetching status for '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-07-07 20:27:45 [INFO ] client#list@1123: Listing '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-07-07 20:27:45 [INFO ] client#list@1123: Listing '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-07-07 20:27:45 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:27:45 [INFO ] client#status@320: Fetching status for '/kafka/topics/

None       id             start_ns          rec_nano_ts       val      ver  \
0     II  1686372038314000000  1686372039283320820  1.149853  v0.0.23   
1     II  1686372038826000000  1686372039623391041  0.070611  v0.0.23   
2     II  1686372038826000000  1686372039963980267  0.471633  v0.0.23   
3     II  1686372039338000000  1686372040304866902  0.387760  v0.0.23   
4     II  1686372039850000000  1686372040645622712  0.322574  v0.0.23   
...   ..                  ...                  ...       ...      ...   
3476  II  1686373175978000000  1686373176977949007 -0.491993  v0.0.23   
3477  II  1686373176490000000  1686373177586859349  0.641429  v0.0.23   
3478  II  1686373177002000000  1686373178182592895  0.716261  v0.0.23   
3479  II  1686373177514000000  1686373178621707140  0.550078  v0.0.23   
3480  II  1686373178026000000  1686373179078254610  2.296289  v0.0.23   

     _device_id  _bed_id   _the_date _hr _patient_id  
0       Procyon  HA11-01  2023-06-10  00     5984929  
1       

In [11]:
df_II_peaks.sort_values(by=['rec_nano_ts'], inplace=True)
df_II_peaks.reset_index(drop=True, inplace=True)
df_II_peaks['shift_rec_nano_ts'] = df_II_peaks['rec_nano_ts'].shift(-1)
df_II_peaks['dif_rec_nano_ts'] = df_II_peaks['shift_rec_nano_ts'] - df_II_peaks['rec_nano_ts']
df_II_peaks;

In [12]:
values_p = np.asarray(df_II_peaks['val'].to_numpy())
raw_time_p = np.asarray(df_II_peaks['rec_nano_ts'].to_numpy(), dtype = 'int')
time_p = [np.datetime64(int(t),'ns') for t in raw_time_p]
time_pks = np.array([(pd.Timestamp(t).hour*60*60*1000) + (pd.Timestamp(t).minute*60*1000) + (pd.Timestamp(t).second*1000) + (pd.Timestamp(t).microsecond/1000) for t in time_p]);

## Pulling continuous signal data from cluster 

In [13]:
kafka_topics_cs = 'nk_waves_NICU'
hdfs_prefix_cs = '/nirscloud/agg_by_hr3/%s'% (kafka_topics_cs)

In [14]:
def _get_df_cs(hdfs_path, the_id):
    hdfs_path = hdfs_path + '/id=%s' % (the_id)
    full_path = nirscloud_util_hdfs.full_path(hdfs_prefix_cs, hdfs_path)
    err, df = nirscloud_util_hdfs.from_hdfs_path(full_path)
    return df

In [15]:
# Time series need to be concatenated accoording to the amount of time that is going to be analyzed 

# Getting a sample from raw data in cluster

##### When getting a sample the first thing to be considered is the amount of time, and this is determined by the stats that are to be done afterwards.
##### We cacn think in terms of how many windows do we want to analyze or how much time, the length of the window and their overlap.

In [16]:
# The total time of the interval will be defined by the amount of windows one chooses, the overlap and the length of the windows.
time_wind_length = 30; overlap_window = 0.5; 
print('Total time length can be in intervals of ' +str(int(time_wind_length*overlap_window))+ ' minutes starting from '+str(time_wind_length))
print('')
print('You can either choose the amount of windows or the total amount of time')

Total time length can be in intervals of 15 minutes starting from 30

You can either choose the amount of windows or the total amount of time


In [17]:
#In this block you can enter the total amount of time to analyze or the amount of windows

by_time = True; total_time = 180; windows = 12


if by_time: 
    by_windows = False
else: by_windows = True

if by_time: 
    total_time = 180
    if total_time%15 != 0: 
        print('The amount of time entered is not divisible by '+str(time_wind_length*overlap_window))
    windows = int(total_time/(time_wind_length*overlap_window)) - 1 
    print('The amount of time is '+str(int(total_time))+' minutes and the amount of windows is '+str(windows))

    
if by_windows: 
    windows = 12
    total_time = (1-overlap_window)*time_wind_length*(windows) + 15
    print('The amount of time is '+str(int(total_time))+' minutes and the amount of windows is '+str(windows))


The amount of time is 180 minutes and the amount of windows is 11


In [18]:
#This line is important because it establishes what will be the first hour 
first_hour = 0

In [19]:
# Extracting and concatenating dataframes according to time. 

if total_time%60 == 0: 
    amount_hours = int(total_time/60)
else:
    amount_hours = int((total_time - total_time%60)/60) + 1 

print('Extracting '+str(int(amount_hours))+' hours')

df_dict = {}; time_cts =[]; values_cs = [];
hours = np.arange(first_hour,first_hour + amount_hours)
lead = 'MDC_ECG_ELEC_POTL_II'

for h in hours:
    if len(str(h)) == 1:
        hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=0' + str(h)+'/_patient_id=5984929'
    else: 
        hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=' + str(h)+'/_patient_id=5984929'

    df_cs = _get_df_cs(hdfs_path, lead)
    print('Extracted hour: '+str(h))
    df_dict['hour '+str(h)] = df_cs
    
final_df_cs = pd.concat(df_dict)


final_df_cs.sort_values(by=['_milli_ts'], inplace=True)
final_df_cs.reset_index(drop=True, inplace=True)
final_df_cs['shift_milli_ts'] = final_df_cs['_milli_ts'].shift(-1)
final_df_cs['diff_milli_ts'] = final_df_cs['shift_milli_ts'] - final_df_cs['_milli_ts']

values_cs = np.asarray(final_df_cs['val'].to_numpy())
raw_time_cs = np.asarray(final_df_cs['_milli_ts'].to_numpy(), dtype = 'int')
time_cs = [np.datetime64(int(t),'ms') for t in raw_time_cs]
time_cts = np.concatenate([time_cts, np.array([(pd.Timestamp(t).hour*60*60*1000) + (pd.Timestamp(t).minute*60*1000) + 
                 (pd.Timestamp(t).second*1000) + (pd.Timestamp(t).microsecond/1000) for t in time_cs])]);


missing_info = final_df_cs[final_df_cs['diff_milli_ts']>4]
missing_ids = missing_info.index.to_list()

if len(missing_ids)>0:
    print('')
    print('There is missing data in the following indices')
for missing_idx in missing_ids: 
    print('')
    print('Time '+str(round((missing_idx*4/1000)/60,2))+', Index '+str(missing_idx)+' is missing '+str(final_df_cs['diff_milli_ts'][missing_idx])+' milliseconds ahead')
    zeros = np.zeros(int(final_df_cs['diff_milli_ts'][missing_idx]/4))
    missing_time = np.arange(time_cts[missing_idx]+4,time_cts[missing_idx+1],4)
    time_cts = np.insert(time_cts,missing_idx+1,missing_time)
    values_cs = np.insert(values_cs,missing_idx,zeros)
    
time_cts = time_cts - time_cts[0]

print('The maximum amount of time is '+ str(int(total_time))+' mins')
print('Finished')


2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#list@1123: Listing '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:

Extracting 3 hours


2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#list@1123: Listing '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:

Extracted hour: 0


2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=02/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=02/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#list@1123: Listing '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=02/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:38 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=02/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-07 20:28:

Extracted hour: 1
Extracted hour: 2

There is missing data in the following indices

Time 96.8, Index 1452031 is missing 260.0 milliseconds ahead
The maximum amount of time is180 mins
Finished


# Classes 

In [20]:
#time_pks = time_pks - time_cts[0]

In [23]:

class raw_sample_():
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label

        self.min_time = int((self.min_min*60*1000)/4) + int(self.min_sec*1000/4)
        self.max_time = int((self.max_min*60*1000)/4) + int(self.max_sec*1000/4) + 1
        self.pks_time_min = (self.min_min*60*1000) + (self.min_sec*1000)
        self.pks_time_max = (self.max_min*60*1000) + (self.max_sec*1000)
        self.pks_time = time_pks[(self.pks_time_min > time_pks).argmin():(time_pks < self.pks_time_max).argmin()]
        self.pks_values = values_p[(self.pks_time_min > time_pks).argmin():(time_pks < self.pks_time_max).argmin()]
        
        self.time =time_cts[self.min_time:self.max_time]
        self.values =values_cs[self.min_time:self.max_time]

        self.fig = go.Figure()
        self.fig.add_trace(go.Scatter(line= dict(color='blue'),x=self.time, y=self.values,showlegend=False))
        self.fig.add_trace(go.Scatter(mode='markers', marker= dict(color='red'),x=self.pks_time, y=self.pks_values,showlegend=False))

        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)


In [20]:

class sample_():
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label,find_the_peaks,inverted):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label

        self.min_time = int((self.min_min*60*1000)/4 + int(self.min_sec*1000)/4) 
        self.max_time = int((self.max_min*60*1000)/4 + int(self.max_sec*1000)/4) + 1

        
        time_zero = raw_time_cs[0]
        self.time = raw_time_cs[self.min_time:self.max_time] - time_zero
        if inverted:
            self.values = - values_cs[self.min_time:self.max_time]
        else: 
            self.values =  values_cs[self.min_time:self.max_time]

        self.fig = go.Figure()
        self.fig.add_trace(
                      go.Scatter(line= dict(color='blue'),x=self.time, y=self.values,showlegend=False))

        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)
        
        if find_the_peaks:
            self.peaks, _ = find_peaks(self.values, distance=100)
            self.fig.add_trace(
                      go.Scatter(mode='markers',x=self.time[self.peaks], y=self.values[self.peaks],showlegend=False))


In [21]:
#bs_r_window = baseline removal window;     #fs = sampling frequency                                      #trans_width = transition width of bandpass 
#numtaps = size of filter                   # mwi_windoow = moving window integration window


class sample_all_included():
    
    first_time = True
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label,inverted, bs_r_window,band_start,band_stop,fs,trans_width,numtaps, 
                 mwi_window, test_segments,spline,res):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label


        self.min_time = int((self.min_min*60*1000)/4 + int(self.min_sec*1000)/4) 
        self.max_time = int((self.max_min*60*1000)/4 + int(self.max_sec*1000)/4) + 1

                
        time_zero = raw_time_cs[0]
        self.time = raw_time_cs[self.min_time:self.max_time] - time_zero
        if inverted:
            self.values = - values_cs[self.min_time:self.max_time]
        else: 
            self.values =  values_cs[self.min_time:self.max_time]
        
       
        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)       
        
        #Filters' Parameters
        self.first_bs_r_window = bs_r_window
        self.first_band = [band_start, band_stop]
        self.first_mwi_window = mwi_window
        self.bs_r_window = bs_r_window
        self.trans_width = trans_width
        self.numtaps = numtaps
        self.fs = fs
        self.band = [band_start,band_stop]
        self.edges = [0, self.band[0] - trans_width, self.band[0], self.band[1], self.band[1] + self.trans_width, 0.5*fs]
        self.test_segments = test_segments
        self.segment_duration = 0.8
        self.spline = spline 
        self.res = res
         
        
        #Bandpass
        self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
        self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
        self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))
        
        for j in range(0,len(self.prepro_data)+len(self.taps)): 
            sum = 0
            for k in range(0,len(self.taps)):
                sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
            self.bp_y[j] = sum
            
        self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
        self.bp_x = self.time[0:len(self.bp_y)]

        
        #derivative
        self.der_y = np.zeros(len(self.bp_y))
        self.der_x = self.bp_x
        self.der_x_in = np.append(self.bp_y,np.zeros(2))

        for i in range(0,len(self.bp_y)):
            self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))
        
        #squaring
        self.sq_y = (self.der_y) ** 2
        self.sq_x = self.der_x
        
        #Moving Window Integration
        self.mwi_window = mwi_window

        
        
    
    @property
    def bs_r_window(self): 
        return self._bs_r_window

    @bs_r_window.setter
    def bs_r_window(self,new_window_value):
        self._bs_r_window = new_window_value
        self.bs_re, self.cut_pts = moving_average(self._bs_r_window,self.values)
        self.prepro_data = self.values[0:len(self.time)-self.cut_pts] - self.bs_re
        self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
        
        if (new_window_value != self.first_bs_r_window): 
            self.first_bs_r_window = 300
            self.bs_re, self.cut_pts = moving_average(self._bs_r_window,self.values)
            self.prepro_data = self.values[0:len(self.time)-self.cut_pts] - self.bs_re
            self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
            
            self.edges = [0, self.band[0] - self.trans_width, self.band[0], self.band[1], self.band[1] + self.trans_width, 0.5*self.fs]
            self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
            self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
            self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))
            
            for j in range(0,len(self.prepro_data)+len(self.taps)): 
                sum = 0
                for k in range(0,len(self.taps)):
                    sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
                self.bp_y[j] = sum

            self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
            self.bp_x = self.time[0:len(self.bp_y)]
            
            #Derivative when bandpass is updated
            self.der_y = np.zeros(len(self.bp_y))
            self.der_x_in = np.append(self.bp_y,np.zeros(2))
            for i in range(0,len(self.bp_y)):
                self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))

            self.der_x = self.bp_x

            #Squaring when bandpass is updatd
            self.sq_y = self.der_y ** 2
            self.sq_x = self.der_x

            #Movig window integgration when bandpass is updated 
            self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
            self.mwi_x = self.der_x

            #Fiducial points when bandpass is updated
            self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
            self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)

            
        
    @property
    def band(self):
        return self._band
    
    @band.setter
    def band(self,new_band):
        self.band_start = new_band[0]; self.band_stop = new_band[1]
        self._band = [new_band[0],new_band[1]]
        if (new_band != self.first_band): 
            self.first_band = 0.01
            self.edges = [0, self._band[0] - self.trans_width, self._band[0], self._band[1], self._band[1] + self.trans_width, 0.5*self.fs]
            self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
            self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
            self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))

            for j in range(0,len(self.prepro_data)+len(self.taps)): 
                sum = 0
                for k in range(0,len(self.taps)):
                    sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
                self.bp_y[j] = sum

            self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
            self.bp_x = self.time[0:len(self.bp_y)]


            #Derivative when bandpass is updated
            self.der_y = np.zeros(len(self.bp_y))
            self.der_x_in = np.append(self.bp_y,np.zeros(2))
            for i in range(0,len(self.bp_y)):
                self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))

            self.der_x = self.bp_x

            #Squaring when bandpass is updatd
            self.sq_y = self.der_y ** 2
            self.sq_x = self.der_x

            #Movig window integgration when bandpass is updated 
            self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
            self.mwi_x = self.der_x

            #Fiducial points when bandpass is updated
            self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
            self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)
                    
    @property
    def mwi_window(self):
        return self._mwi_window
    
    @mwi_window.setter
    def mwi_window(self,new_window):
        self._mwi_window = new_window
        self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
        self.mwi_x = self.der_x
        self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
        self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)
    
    

        
    
        #self.prepro_fig = go.Figure()
        #self.prepro_fig.add_trace(go.Scatter(line=dict(color='orange'),x = self.prepro_x, y = self.prepro_data),showlegend = show_legend)



### Functions

## Cluster Sample

In [24]:
cluster_sample = raw_sample_(1,30,3,0,1)
fig_raw = cluster_sample.fig
fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1400,
                  height=600,showlegend=False,title="Raw Data ", xaxis_title="Time", yaxis_title= 'Signal', 
                  font=dict(family="Avenir",size=14,color="Black"));

## Pan Tompkins

In [25]:
# Sample One: sample_test

In [28]:
min_min_t = 0; min_sec_t = 0; max_min_t= 20; max_sec_t = 0; seconds_per_label_t = 15; inverted_t = False;   bs_r_window_t = 6; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

sample_test = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

/home/shared/braulio/ecg_algorithm/functions.py:232: RuntimeWarning:

invalid value encountered in sqrt

/home/shared/braulio/ecg_algorithm/functions.py:233: RuntimeWarning:

invalid value encountered in sqrt



In [22]:
# Sample Two: sample_test_dif_wl    this sample will have different window length for the mean average subtraction

In [23]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 10; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

sample_test_dif_wl = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

In [29]:
fig_raw = make_subplots(rows=3, cols=1, subplot_titles=("Peaks ", 'Mean Subtraction', 'Moving Window Integration'))

fig_raw.add_trace(go.Scatter(line = dict(color = 'blue'), x = sample_test.time,y = sample_test.values,showlegend=False),row=1, col=1)
fig_raw.add_trace(go.Scatter(mode = 'markers', x = sample_test.max_x,y = sample_test.max_y,showlegend=False),row=1, col=1)
fig_raw.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=2,col=1)
fig_raw.add_trace(go.Scatter(line=dict(color='#0a4345'),x = sample_test.mwi_x,y = sample_test.mwi_y,showlegend=False),row=3,col=1)
fig_raw.add_trace(go.Scatter(marker=dict(color='red'),mode ='markers',x = sample_test.mwi_x[sample_test.fiducial_points],y = sample_test.mwi_y[sample_test.fiducial_points],
                              showlegend=False),row=3,col=1)



fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                    xaxis3 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis))

fig_raw.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000],xaxis3_range=[79000,85000],yaxis_range=[-1.5,1.3],
                       yaxis2_range=[-0.4,0.4], yaxis3_range=[-0.001,0.05])
fig_raw.update_layout(height=1200,width = 1200);

In [31]:
#fig_raw.show()

## Mising Peaks

In [26]:
missing_pt_p, missing_cluster_p = find_missing_peaks(sample_test.max_x, cluster_sample.pks_time, True)


In [27]:
fig_c_vs_s =go.Figure()

fig_c_vs_s.add_trace(go.Scatter(line= dict(color='blue'),x=cluster_sample.time, y=cluster_sample.values,name='Raw Data',showlegend=True))
    

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=cluster_sample.pks_time, y= cluster_sample.pks_values, name = 'Cluster', 
                           showlegend=True,))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test.max_x, 
                          y = sample_test.max_y, name = 'Current Version', showlegend=True))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test.max_x[missing_cluster_p], 
                           y= sample_test.max_y[missing_cluster_p], name = 'Missing Cluster Peaks', showlegend=True))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = cluster_sample.pks_time[missing_pt_p] , y= cluster_sample.pks_values[missing_pt_p], 
                          name = 'Missing Current Version Peaks', showlegend=True))


fig_c_vs_s.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=630,
                    title='Missing Cluster Peaks: '+str(len(missing_cluster_p))+'           Missing Current Version Peaks: '+str(len(missing_pt_p)) + 
                   '            MAW: '+str(sample_test.bs_r_window) + '         Band: '+str(sample_test.band_start)+' - '+str(sample_test.band_stop));


In [28]:
#fig_c_vs_s.show()

##

## Comparing different window lengths against 6 window length in mean average subtraction

In [29]:
#Visually missing peaks

fig_ms_p = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    missing_6_p, missing_dif_wl_p = find_missing_peaks(sample_test.max_x, sample_test_dif_wl.max_x, True)
    missing_6_dict['window_length_'+str(N)] = missing_6_p
    missing_dif_wl_dict['window_length_'+str(N)] = missing_dif_wl_p
    
    fig_ms_p.add_trace(go.Scatter(line= dict(color='blue'),x=sample_test.time, y=sample_test.values,name='Raw Data',showlegend=True, visible = False))
    

    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=sample_test.max_x, y= sample_test.max_y, name = '6 WL Peaks', 
                               showlegend=True, visible = False))
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test_dif_wl.max_x, y = sample_test_dif_wl.max_y,
                               name = str(N)+' WL Peaks',showlegend=True, 
                              visible = False))
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test_dif_wl.max_x[missing_6_p], 
                               y= sample_test_dif_wl.max_y[missing_6_p], name = '6 WL Missing Peaks', showlegend=True, visible = False))
    
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = sample_test.max_x[missing_dif_wl_p] , y= sample_test.max_y[missing_dif_wl_p], 
                              name = str(N)+' WL Missing  Peaks', showlegend=True, visible = False))

fig_ms_p.data[5].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_ms_p.data)},
              {"title": "Peaks identified with window length "+str(lengths[i])+ '   Amount of 6 WL Missing Peaks: '+
               str(len(missing_6_dict['window_length_'+str(lengths[i])])) +'    Amount of ' + str(lengths[i]) + ' WL Missing Peaks: '+ 
               str(len(missing_dif_wl_dict['window_length_'+str(lengths[i])]))}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True
    step["args"][0]["visible"][start+2] = True
    step["args"][0]["visible"][start+3] = True
    step["args"][0]["visible"][start+4] = True
    
    start = start+5
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_ms_p.update_layout(sliders=sliders);
fig_ms_p.update_layout(xaxis_range=[79000,85000]);
fig_ms_p.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

/home/shared/braulio/ecg_algorithm/functions.py:232: RuntimeWarning:

invalid value encountered in sqrt

/home/shared/braulio/ecg_algorithm/functions.py:233: RuntimeWarning:

invalid value encountered in sqrt



In [30]:
#fig_ms_p.show()

## Pre Processed data with different window lengths

In [31]:

fig_prepro = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    fig_prepro.add_trace(go.Scatter(x = sample_test_dif_wl.prepro_x , 
                              y= sample_test_dif_wl.prepro_data, name = str(N)+' WL Prepro Data', showlegend=True, visible = False))

fig_prepro.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_prepro.data)},
              {"title": 'Preprocessed data, window length: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_prepro.update_layout(sliders=sliders);
fig_prepro.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

In [32]:
#fig_prepro.show()

## Bandpass 

In [33]:

figbp = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    figbp.add_trace(go.Scatter(x = sample_test_dif_wl.bp_x , 
                              y= sample_test_dif_wl.bp_y, name = str(N)+' WL in MS, Bandpass', showlegend=True, visible = False))

figbp.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(figbp.data)},
              {"title": 'Bandpass , window length in mean subtraction: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
figbp.update_layout(sliders=sliders);
figbp.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

In [34]:
#figbp.show()

## Derivative

In [35]:

fig_der = go.Figure()
missing_6_dict ={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    fig_der.add_trace(go.Scatter(x = sample_test_dif_wl.der_x , 
                              y= sample_test_dif_wl.der_y, name = str(N)+' WL in MS, Derivative', showlegend=True, visible = False))

fig_der.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_der.data)},
              {"title": 'Derivative , window length in mean subtraction: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_der.update_layout(sliders=sliders);
fig_der.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),autosize=False,width=1200,height=600); 

In [36]:
#fig_der.show()

## Moving Window Integration + Fiducial Points

In [37]:
mwi_fp = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    mwi_fp.add_trace(go.Scatter(x = sample_test_dif_wl.mwi_x , 
                              y= sample_test_dif_wl.mwi_y, name = str(N)+' Bandpass WL: '+str(N), showlegend=True, visible = False))
    mwi_fp.add_trace(go.Scatter(mode = 'markers', x = sample_test_dif_wl.mwi_x[sample_test_dif_wl.fiducial_points] , 
                              y= sample_test_dif_wl.mwi_y[sample_test_dif_wl.fiducial_points], name = str(N)+' Bandpass WL: '+str(N), 
                              showlegend=True, visible = False))

mwi_fp.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(mwi_fp.data)},
              {"title": 'Bandpass data, window length: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start + 1] = True  # Toggle i'th trace to "visible"
    start = start + 2
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
mwi_fp.update_layout(sliders=sliders);
mwi_fp.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

In [38]:
#mwi_fp.show()

In [39]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 15; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

recreated_cluster = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

### Recreating cluster sample 

In [40]:
comp_prepro = make_subplots(rows=2, cols=1, subplot_titles=("Cluster ", 'MWI: '+str(sample_test.bs_r_window)))
comp_prepro.add_trace(go.Scatter(x = recreated_cluster.prepro_x,y = recreated_cluster.prepro_data,showlegend=False),row=1, col=1)
comp_prepro.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=2, col=1)

comp_prepro.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                          xaxis2 = dict(tickmode = 'array', tickvals = recreated_cluster.ticks_values, ticktext = recreated_cluster.time_format_axis))
comp_prepro.update_layout(yaxis_range=[-1.2,1.2],yaxis2_range=[-0.45,0.4])
comp_prepro.update_layout(height=900,width = 1200);

In [41]:
sample_test.band=[4,100]
missing_pt_p, missing_cluster_p = find_missing_peaks(sample_test.max_x, cluster_sample.pks_time, True)
comp_bandpass = make_subplots(rows=3, cols=1, subplot_titles=('Peaks','MWI: '+str(sample_test.bs_r_window) , 'Cluster'))
comp_bandpass.add_trace(go.Scatter(x=cluster_sample.time, y= cluster_sample.values, name = 'Cluster', showlegend=True,),row = 1, col=1)
comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=cluster_sample.pks_time, y= cluster_sample.pks_values, name = 'Cluster', 
                           showlegend=True,),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test.max_x, 
                          y = sample_test.max_y, name = 'Current Version', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test.max_x[missing_cluster_p], 
                           y= sample_test.max_y[missing_cluster_p], name = 'Missing Cluster Peaks', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = cluster_sample.pks_time[missing_pt_p] , y= cluster_sample.pks_values[missing_pt_p], 
                          name = 'Missing Current Version Peaks', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(x = sample_test.bp_x,y = sample_test.bp_y,showlegend=False),row=2, col=1)
comp_bandpass.add_trace(go.Scatter(x = recreated_cluster.bp_x,y = recreated_cluster.bp_y,showlegend=False),row=3, col=1)

comp_bandpass.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),
                            xaxis2 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                            xaxis3 = dict(tickmode = 'array', tickvals = recreated_cluster.ticks_values, ticktext = recreated_cluster.time_format_axis))
comp_bandpass.update_layout(xaxis_range=[141300,142900],xaxis2_range=[141300,142900],yaxis_range=[-2.8,3],yaxis2_range=[-0.2,0.2])
comp_bandpass.update_layout(height=1200,width = 1200);